In [ ]:
# Initialisation
lpse_utils_loc = '..'
import sys; sys.path.insert(0,lpse_utils_loc)
import lpse_utils.write_files as wf
from lpse_utils.extract_data import get_metrics,get_fields
import numpy as np
%load_ext autoreload
%autoreload 2
!{'> lpse.parms'}
!{'mkdir data'}
show_atts = True

LPSE user guide contains additional options to those listed

# Job Control

In [ ]:
# Job control file options
jc = wf.job_control()
jc.version = '3.2.11'
jc.verbose = 2
jc.seed = 1 # 0 for random, otherwise fixed seed
jc.resources.heartbeatInterval = 0.1 # minutes
jc.write(show_atts)

# Gridding

In [ ]:
# Gridding file options
gr = wf.gridding()
gr.grid.sizes = 100 # microns
gr.grid.nodes = 4000
gr.grid.antiAliasing.range = 0.333
gr.grid.antiAliasing.isAutomatic = 'false'
gr.write(show_atts)

# Components

In [ ]:
cm = wf.components()
cm.laser.enable = 'true'
cm.raman.enable = 'true'
cm.lw.enable = 'true'
cm.iaw.enable = 'false'
cm.hpe.enable = 'false'
cm.write(show_atts)

# Temporal Control

In [ ]:
tc = wf.temporal_control()
tc.simulation.samplePeriod = 0.05 # ps
tc.simulation.time.end = 2 # ps
tc.write(show_atts)

# I/O Control

In [ ]:
# Specifying a filename turns on output
io = wf.io_control()
io.grid.downSampleFactors = 4 # Spatial downsampling per dimension
fp = './data/lpse.' # File prefix
io.lw.save.pots = fp + 'pots'
io.laser.save.E0.z = fp + 'E0_z'
io.laser.save.S0.x = fp + 'S0_x'
io.raman.save.E0.z = fp + 'E1_z'
io.raman.save.S0.x = fp + 'S1_x'
io.raman.save.E0.__dict__['2'] = None
io.write(show_atts)
field_fnames = io.fnames()

# Physical Parameters

In [ ]:
pp = wf.physical_parameters()
pp.physical.Z = 3
pp.physical.Te = 0.1 # keV
pp.physical.Ti = 0.1 # keV
pp.physical.MiOverMe = 1836.15
pp.lw.envelopeDensity = 0.152
pp.densityProfile.shape = 'linear'
pp.densityProfile.geometry = 'cartesian'
pp.densityProfile.NminOverNc = 0.145
pp.densityProfile.NmaxOverNc = 0.155
pp.densityProfile.NminLocation = '-50 0 0'
pp.densityProfile.NmaxLocation = '50 0 0'
pp.write(show_atts)

# Light Control

In [ ]:
lc = wf.light_control()
lc.laser.pumpDepletion.SRS.enable = 'true'
lc.laser.ionAcousticPerturbations.enable = 'false'
lc.laser.wavelength = 0.351
lc.laser.solver = 'fd'
lc.laser.evolution.dtFraction = 0.95
lc.laser.evolution.Labc = 3 # microns
lc.laser.evolution.abc.type = 'pml'
lc.laser.evolution.Loff = 1 # microns
lc.laser.evolution.absorption = 0
lc.raman.sourceTerm.lw.enable = 'true'
lc.raman.ionAcousticPerturbations.enable = 'false'
lc.raman.solver = 'fd'
lc.raman.evolution.dtFraction = 0.95
lc.raman.evolution.Labc = 3
lc.raman.evolution.abc.type = 'pml'
lc.raman.evolution.absorption = 0
lc.write(show_atts)

# Light Sources

In [ ]:
ls = wf.light_source()
ls.laser.nBeams = 1
ls.laser.intensity = ['2e+14'] # W/cm^2
ls.laser.phase = [0] # degrees
ls.laser.polarization = [90] # degrees
ls.laser.direction = ['1 0 0']
ls.laser.frequencyShift = [0]
ls.laser.group = [0]
ls.laser.evolution.source = ['min.x']
ls.laser.evolution.offset = ['0 0 0'] # microns
ls.laser.evolution.width = [10] # Half-width at 1/e of sgauss [um]
ls.laser.evolution.sgOrder = [4]
ls.raman.nBeams = 1
ls.raman.intensity = ['1e+12'] # W/cm^2
ls.raman.phase = [0] # degrees
ls.raman.polarization = [90] # degrees
ls.raman.direction = ['-1 0 0']
ls.raman.frequencyShift = [0]
ls.raman.group = [0]
ls.raman.evolution.source = ['max.x']
ls.raman.evolution.offset = ['0 0 0'] # microns
ls.raman.evolution.width = [10] # Half-width at 1/e of sgauss [um]
ls.raman.evolution.sgOrder = [4] 
ls.write(show_atts)

# Langmuir Wave Parameters

In [ ]:
lwc = wf.lw_control()
lwc.lw.SRS.enable = 'true'
lwc.lw.spectral.dt = 0.002 # ps
lwc.lw.maxLightStepsPerStep = 10
lwc.lw.Labc = 3 # microns
lwc.lw.noise.enable = 'false'
lwc.lw.noise.isCalculated = 'false'
lwc.lw.noise.amplitude = 1.67772
lwc.lw.collisionalDampingRate = 0.1
lwc.lw.__dict__['collisionalDampingRate.isCalculated'] = 'false'
lwc.write(show_atts)

# Ion Acoustic Wave Parameters

In [ ]:
iawc = wf.iaw_control()
iawc.write(show_atts)

# Instrumentation

In [ ]:
ins = wf.instrumentation()
ins.metrics.enable = 'true'
ins.metrics.file = fp + 'metrics'
ins.metrics.samplePeriod = 0.01 # ps
ins.write(show_atts)

# Run case

In [ ]:
nprocs = 1
lpse_binary_loc = '../../bin/lpse_cpu'
!{'mpirun -np ' + str(nprocs) + ' ' + lpse_binary_loc + ' --parms=lpse.parms'}

# Visualise and extract datasets

In [ ]:
# Extract metrics data
mkeys, mdat = get_metrics(plot=True)
print(mkeys)

In [ ]:
# Extract fields data
innerkys = ['time','x','data']
outerkys = [field_fnames[i].replace(fp,'') \
            for i in range(len(field_fnames))]
field_dicts = {i:{j:None for j in innerkys} for i in outerkys}
for i in range(len(field_fnames)):
    field_dicts[outerkys[i]] = \
      get_fields(field_fnames[i],field_dicts[outerkys[i]],\
                 io.grid.downSampleFactors,plot=True)

In [ ]:
# Laser light transmission at final timestep
icell = 50
ocell = -50
linx = field_dicts['S0_x']['x'][icell]
lin = np.real(field_dicts['S0_x']['data'][-1,icell]) 
loutx = field_dicts['S0_x']['x'][ocell]
lout = np.real(field_dicts['S0_x']['data'][-1,ocell])
enl = 1-lout/lin
print(f'Laser inner Poynting flux (at {linx:0.1f} um)' \
      + f' at sim end: {lin:0.3e} W/cm^2')
print(f'Laser outer Poynting flux (at {loutx:0.1f} um)' \
      + f' at sim end: {lout:0.3e} W/cm^2')
print(f'This is an energy loss of {enl:.2%}')

In [ ]:
# Raman light energy gain at final timestep
icell = -50
ocell = 50
rinx = field_dicts['S1_x']['x'][icell]
rin = np.real(field_dicts['S1_x']['data'][-1,icell]) 
routx = field_dicts['S1_x']['x'][ocell]
rout = np.real(field_dicts['S1_x']['data'][-1,ocell])
eng = rout/rin - 1
lenl = abs(rout-rin)/(lin-lout)
print(f'Raman inner Poynting flux (at {rinx:0.1f} um)' \
      + f' at sim end: {rin:0.3e} W/cm^2')
print(f'Raman outer Poynting flux (at {routx:0.1f} um)' \
      + f' at sim end: {rout:0.3e} W/cm^2')
print(f'This is an energy gain of {eng:.2%}')
print(f'It represents {lenl:.2%} of the laser energy loss')